Imports and functions

In [373]:
import os
import netCDF4 as nc4
import numpy as np
#import numpy.ma as ma

User inputs

In [374]:
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/06/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/07/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/10/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/11/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/12/test'
#-----------------------------------------------------------------
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/02/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/04/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/06/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/07/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/08/test'
#-----------------------------------------------------------------
indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2016/01/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2016/06/test'

THRESHOLD = 38.0  #dBZ
SLOPE_LO = 6.0
SLOPE_HI2 = 8.0
SLOPE_HI3 = 8.0
VERT_RES = 0.125  #km

In [375]:
#fname = 'GPM-2Ku.001497.20140603.213419.nc'
#fname = 'GPM-2Ku.001574.20140608.202205.nc'
#fname = 'GPM-2Ku.001779.20140622.005639.nc'
#fname = 'GPM-2Ku.001917.20140630.215001.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.002081.20140711.102925.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.003485.20141009.163649.nc'
#fname = 'GPM-2Ku.003510.20141011.064348.nc'
#fname = 'GPM-2Ku.003556.20141014.053954.nc'
#fname = 'GPM-2Ku.003684.20141022.113356.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.003886.20141104.110624.nc'
#fname = 'GPM-2Ku.004007.20141112.054200.nc'
#fname = 'GPM-2Ku.004022.20141113.044926.nc'
#fname = 'GPM-2Ku.004186.20141123.171346.nc'
#fname = 'GPM-2Ku.004201.20141124.162249.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.004378.20141206.014736.nc'
#fname = 'GPM-2Ku.004665.20141224.120452.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.005285.20150202.084127.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.006190.20150401.125219.nc'
#fname = 'GPM-2Ku.006329.20150410.111317.nc'
#fname = 'GPM-2Ku.006437.20150417.094529.nc'
#fname = 'GPM-2Ku.006569.20150425.205420.nc'
#fname = 'GPM-2Ku.006621.20150429.053449.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.007400.20150618.065614.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.007789.20150713.071724.nc'
#fname = 'GPM-2Ku.007866.20150718.060539.nc'
#fname = 'GPM-2Ku.008020.20150728.034222.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.008343.20150817.220244.nc'
#-------------------------------------------
fname = 'GPM-2Ku.010496.20160103.071812.nc'
#-------------------------------------------
#fname = 'GPM-2Ku.013031.20160614.053444.nc'
#fname = 'GPM-2Ku.013087.20160617.194203.nc'
#fname = 'GPM-2Ku.013200.20160625.021853.nc'   

In [376]:
ncid = nc4.Dataset(indir+'/'+fname,'a')
#ncid = nc4.Dataset(indir+'/'+fname,'r')
    
refl = np.array(ncid.variables['refl'])
(ntime,nalt,nlat,nlon) = refl.shape
refl_missing = ncid.variables['refl'].missing_value
bin_missing = int(refl_missing)
    
rt = np.array(ncid.variables['rain_type'])
rt_missing = ncid.variables['rain_type'].missing_value
#rt[(rt==rt_missing)] = np.nan 
    
rt_uw2 = np.array(rt, copy=True)    
rt_uw3 = np.array(rt, copy=True)    

    For rt_uw2, change rain_type to stratiform in rt_uw if:
    1. rain_type = convective and
    2. max_refl in column >= THRESHOLD and
    3. (max_refl - dbz@(max_refl_ht-1km)/deltaHt <= abs(SLOPE_LO)
    4. (max_refl - dbz@(max_refl_ht+1km)/deltaHt >= abs(SLOPE_HI)
    
    For rt_uw3, change rain_type to stratiform in rt_uw if:
    1. rain_type = convective and
    2. max_refl in column >= THRESHOLD and
    3. (max_refl - dbz@(max_refl_ht+1km)/deltaHt >= abs(SLOPE_HI)    

In [377]:
# find max_refl and max_refl_ht at each lat/lon
max_refl = np.zeros(rt.shape,dtype=float)+refl_missing
max_refl = np.amax(refl,axis=1)
max_refl_bin = np.zeros(rt.shape,dtype=int)+bin_missing
max_refl_bin = np.argmax(refl,axis=1)
max_refl_ht = max_refl_bin * VERT_RES

max_refl_bin[(max_refl==refl_missing)] = bin_missing
max_refl_ht[(max_refl==refl_missing)] = refl_missing
    
# find bin & height of highest good refl at each lat/lon
max_ht_bin = np.zeros(rt.shape,dtype=int) + bin_missing
for lev in range(nalt-1,-1,-1):
    max_ht_bin = np.where(np.logical_and((refl[:,lev,:,:] != refl_missing),
                                        (max_ht_bin == bin_missing) ), 
                        lev, max_ht_bin ) 
max_ht = max_ht_bin * (max_ht_bin != bin_missing) * VERT_RES
max_ht[(max_ht_bin==bin_missing)] = refl_missing    
    
# find bin & height of lowest good refl at each lat/lon
min_ht_bin = np.zeros(rt.shape,dtype=int) + bin_missing
for lev in range(0,nalt):
    min_ht_bin = np.where(np.logical_and( (refl[:,lev,:,:] != refl_missing),
                                    (min_ht_bin == bin_missing) ), 
                        lev, min_ht_bin ) 
min_ht = min_ht_bin * (min_ht_bin != bin_missing) * VERT_RES
min_ht[(min_ht_bin==bin_missing)] = refl_missing    

In [378]:
# Determine indices of interest 
change = np.logical_and(rt==2,max_refl>=THRESHOLD)
ind_time,ind_lat,ind_lon = np.where(change==True)
npixels = ind_time.size

In [379]:
# Run test for rt_uw2 and rt_uw3
for ind in range(0,npixels):
    itime = ind_time[ind]
    ilat = ind_lat[ind]
    ilon = ind_lon[ind]
    #itime = 0
    # Problem point with max refl at 0.125km
    #ilat = 266
    #ilon =  24
    # Good point with max refl at 1.75km
    #ilat = 265
    #ilon =  34
    # Fails at SLOPE_HI=8, passes at SLOPE_HI=6
    #ilat = 264
    #ilon = 25
    #ilon = 100
    #ilat = 45
    #print 'ind = ',ind,' and [itime,ilat,ilon] = [',itime,',',ilat,',',ilon,'] and max_refl = ',max_refl[itime,ilat,ilon]
    if max_refl[itime,ilat,ilon] != refl_missing:
        #print 'max_refl_bin = ',max_refl_bin[itime,ilat,ilon],' and max_refl = ',max_refl[itime,ilat,ilon]
        #print 'Looking for minus point'
        if max_refl_bin[itime,ilat,ilon] >= int(1.0/VERT_RES):
            #print '   Not atypical'
            ht_index = max_refl_bin[itime,ilat,ilon]-int(1.0/VERT_RES)
            #print '   ht_index = ',ht_index,' and refl = ',refl[itime,ht_index,ilat,ilon]
            refl_minus_diff = max_refl[itime,ilat,ilon]-refl[itime,ht_index,ilat,ilon]
            ht_minus_diff = 1.0
            #print '   refl_minus_diff = ',refl_minus_diff,' and ht_minus_diff = ',ht_minus_diff
        else:
            #print '   Atypical'
            ht_index = min_ht_bin[itime,ilat,ilon]
            #print '   ht_index = ',ht_index,' and refl = ',refl[itime,ht_index,ilat,ilon]
            refl_minus_diff = max_refl[itime,ilat,ilon]-refl[itime,ht_index,ilat,ilon]
            ht_minus_diff = max_refl_ht[itime,ilat,ilon]-min_ht[itime,ilat,ilon]
            #print '   refl_minus_diff = ',refl_minus_diff,' and ht_minus_diff = ',ht_minus_diff
        if ht_minus_diff != 0.0:
            slope_minus = refl_minus_diff/ht_minus_diff
        else:
            slope_minus = refl_missing
        #print '   slope_minus = ',slope_minus
        
        #print 'Looking for plus point'
        if max_ht_bin[itime,ilat,ilon] >= max_refl_bin[itime,ilat,ilon]+int(1.5/VERT_RES):
            #print '   Not atypical'
            ht_index = max_refl_bin[itime,ilat,ilon]+int(1.5/VERT_RES)
            #print '   ht_index = ',ht_index,' and refl = ',refl[itime,ht_index,ilat,ilon]
            refl_plus_diff = max_refl[itime,ilat,ilon]-refl[itime,ht_index,ilat,ilon]
            ht_plus_diff = 1.5
            #print '   refl_plus_diff = ',refl_plus_diff,' and ht_plus_diff = ',ht_plus_diff
        else:
            #print '   Atypical'
            ht_index = max_ht_bin[itime,ilat,ilon]
            #print '   ht_index = ',ht_index,' and refl = ',refl[itime,ht_index,ilat,ilon]
            refl_plus_diff = max_refl[itime,ilat,ilon]-refl[itime,ht_index,ilat,ilon]
            #print 'refl = ',
            ht_plus_diff = max_ht[itime,ilat,ilon]-max_refl_ht[itime,ilat,ilon]
            #print '   refl_plus_diff = ',refl_plus_diff,' and ht_plus_diff = ',ht_plus_diff
        if ht_plus_diff != 0.0:
            slope_plus = refl_plus_diff/ht_plus_diff
        else:
            slope_plus = refl_missing
        #print '   slope_plus = ',slope_plus
                
        change2 = np.logical_and(abs(slope_minus) <= SLOPE_LO,
                                abs(slope_plus) >= SLOPE_HI2)
        #print 'change2 = ',change2
        if change2 == True:
            rt_uw2[itime,ilat,ilon] = 1
                
        change3 = abs(slope_plus) >= SLOPE_HI3
        #print 'change3 = ',change3
        if change3 == True:
            rt_uw3[itime,ilat,ilon] = 1

    Add new raintype fields to input netcdf file

In [235]:
rt_uw2_id = ncid.createVariable('rain_type_uw2','f4',
                                ('time','latitude','longitude'),zlib=True)
rt_uw2_id.units = "none"
rt_uw2_id.long_name = "Rain Type UW2"
rt_uw2_id.stratiform = 1
rt_uw2_id.convective = 2
rt_uw2_id.other = 3
rt_uw2_id.missing_value = rt_missing
rt_uw2_id[:,:,:] = rt_uw2

In [380]:
rt_uw3_id = ncid.createVariable('rain_type_uw3','f4',
                                ('time','latitude','longitude'),zlib=True)
rt_uw3_id.units = "none"
rt_uw3_id.long_name = "Rain Type UW3"
rt_uw3_id.stratiform = 1
rt_uw3_id.convective = 2
rt_uw3_id.other = 3
rt_uw3_id.missing_value = rt_missing
rt_uw3_id[:,:,:] = rt_uw3

In [381]:
ncid.close()